In [1]:
from datasets import load_dataset

In [2]:
raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

/Users/hippolyteleveque/Documents/learning_projects/ML/huggingface-tutorials/env/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
split_datasets = raw_datasets["train"].train_test_split(train_size=.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [4]:
split_datasets["validation"] = split_datasets.pop("test")

In [5]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [6]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expand threads")

/Users/hippolyteleveque/Documents/learning_projects/ML/huggingface-tutorials/env/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour étendre les threads'}]

In [7]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [8]:
found = None
for el in split_datasets["train"]["translation"]:
    if "email" in el["en"]:
        found = el
        break

In [9]:
found

{'en': 'Sends the chart as an email attachment.',
 'fr': 'Envoie le diagramme comme pièce jointe.'}

In [10]:
def extract_languages(examples):
    translations = examples["translation"]
    return {"en": [ example["en"] for example in translations], "fr": [ example["fr"] for example in translations]}

In [11]:
new_dataset = split_datasets.map(extract_languages, batched=True, remove_columns=["translation"])

In [12]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [13]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)

In [14]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [15]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [16]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [19]:
batch["labels"]

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [20]:
batch["input_ids"]

tensor([[47591,    12,  9842, 19634,     9,     0, 59513, 59513, 59513, 59513,
         59513, 59513, 59513, 59513, 59513],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
         28149,   139, 33712, 25218,     0]])

In [21]:
batch["decoder_input_ids"]

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [22]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


In [23]:
!pip install sacrebleu

In [24]:
!pip install evaluate

In [25]:
import evaluate

metric = evaluate.load("sacrebleu")

In [26]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [27]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [28]:
from huggingface_hub import notebook_login

notebook_login()

In [31]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    #fp16=True,
    push_to_hub=True,
)

In [32]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [34]:
#trainer.evaluate(max_length=max_length)

In [35]:
#trainer.train()

In [36]:
#from torch.utils.data import DataLoader
#
#tokenized_datasets.set_format("torch")
#train_dataloader = DataLoader(
#    tokenized_datasets["train"],
#    shuffle=True,
#    collate_fn=data_collator,
#    batch_size=8,
#)
#eval_dataloader = DataLoader(
#    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
#)

In [37]:
#model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [39]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [40]:
#from accelerate import Accelerator
#
#accelerator = Accelerator()
#model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
#    model, optimizer, train_dataloader, eval_dataloader
#)

In [41]:
#from transformers import get_scheduler
#
#num_train_epochs = 3
#num_update_steps_per_epoch = len(train_dataloader)
#num_training_steps = num_train_epochs * num_update_steps_per_epoch
#
#lr_scheduler = get_scheduler(
#    "linear",
#    optimizer=optimizer,
#    num_warmup_steps=0,
#    num_training_steps=num_training_steps,
#)

In [42]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [43]:
#from tqdm.auto import tqdm
#import torch
#
#progress_bar = tqdm(range(num_training_steps))
#
#for epoch in range(num_train_epochs):
#    # Training
#    model.train()
#    for batch in train_dataloader:
#        outputs = model(**batch)
#        loss = outputs.loss
#        accelerator.backward(loss)
#
#        optimizer.step()
#        lr_scheduler.step()
#        optimizer.zero_grad()
#        progress_bar.update(1)
#
#    # Evaluation
#    model.eval()
#    for batch in tqdm(eval_dataloader):
#        with torch.no_grad():
#            generated_tokens = accelerator.unwrap_model(model).generate(
#                batch["input_ids"],
#                attention_mask=batch["attention_mask"],
#                max_length=128,
#            )
#        labels = batch["labels"]
#
#        # Necessary to pad predictions and labels for being gathered
#        generated_tokens = accelerator.pad_across_processes(
#            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
#        )
#        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
#
#        predictions_gathered = accelerator.gather(generated_tokens)
#        labels_gathered = accelerator.gather(labels)
#
#        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
#        metric.add_batch(predictions=decoded_preds, references=decoded_labels)
#
#    results = metric.compute()
#    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")
#
#    # Save and upload
#    accelerator.wait_for_everyone()
#    unwrapped_model = accelerator.unwrap_model(model)
#    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
#    if accelerator.is_main_process:
#        tokenizer.save_pretrained(output_dir)
#        repo.push_to_hub(
#            commit_message=f"Training in progress epoch {epoch}", blocking=False
#        )